In [1]:
!pip install wordcloud

Defaulting to user installation because normal site-packages is not writeable


In [2]:
pip install pymongo

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
from wordcloud import WordCloud
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
# !pip install neattext
from keras.models import load_model
import neattext.functions as nfx
import matplotlib.pyplot as plt
import plotly.express as plx
from sklearn.metrics import classification_report
import keras
from keras.layers import Embedding,Dense,LSTM,Bidirectional,GlobalMaxPooling1D,Input,Dropout
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.models import Sequential
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer

In [2]:
from keras.utils import pad_sequences
from tqdm import tqdm
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')

In [3]:
data=pd.read_csv('short.csv')
data.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,text,class
0,1,1,3,Am I weird I don't get affected by compliments...,non-suicide
1,2,2,4,Finally 2020 is almost over... So I can never ...,non-suicide
2,3,3,8,i need helpjust help me im crying so hard,suicide
3,4,4,9,Oh Fuck! can't take this anymore.I am tired of...,suicide
4,5,5,11,I am gonna hurt myself,suicide


In [5]:
data=data.drop(["Unnamed: 0.1","Unnamed: 0"], axis=1)
data.head()

KeyError: "['Unnamed: 0.1', 'Unnamed: 0'] not found in axis"

In [9]:
data['class'].value_counts()

non-suicide    55200
suicide        26442
Name: class, dtype: int64

In [10]:
data['class'].value_counts().index.values

array(['non-suicide', 'suicide'], dtype=object)

In [11]:
train_data,test_data=train_test_split(data,test_size=0.2,random_state=10)

In [12]:
train_data['class'].value_counts().index.values

array(['non-suicide', 'suicide'], dtype=object)

In [13]:

def clean_text(text):
    text_length=[]
    cleaned_text=[]
    for sent in tqdm(text):
        sent=sent.lower()
        sent=nfx.remove_special_characters(sent)
        sent=nfx.remove_stopwords(sent)
#         sent=nfx.remove_shortwords(sent)
        text_length.append(len(sent.split()))
        cleaned_text.append(sent)
    return cleaned_text,text_length

In [14]:
cleaned_train_text,train_text_length=clean_text(train_data.text)
cleaned_test_text,test_text_length=clean_text(test_data.text)

100%|██████████| 16329/16329 [00:00<00:00, 57682.15it/s]


In [15]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(cleaned_train_text)

In [16]:
word_freq=pd.DataFrame(tokenizer.word_counts.items(),columns=['word','count']).sort_values(by='count',ascending=False)

In [17]:
train_text_seq=tokenizer.texts_to_sequences(cleaned_train_text)
train_text_pad=pad_sequences(train_text_seq,maxlen=50)


test_text_seq=tokenizer.texts_to_sequences(cleaned_test_text)
test_text_pad=pad_sequences(test_text_seq,maxlen=50)

In [14]:
lbl_target=LabelEncoder()
train_output=lbl_target.fit_transform(train_data['class'])
test_output=lbl_target.transform(test_data['class'])

In [14]:
# glove_embedding={}
with open('glove.840B.300d.pkl', 'rb') as fp:
    glove_embedding = pickle.load(fp)

In [21]:
v=len(tokenizer.word_index)

embedding_matrix=np.zeros((v+1,300), dtype=float)
for word,idx in tokenizer.word_index.items():
    embedding_vector=glove_embedding.get(word)
    if embedding_vector is not None:
        embedding_matrix[idx]=embedding_vector

NameError: name 'glove_embedding' is not defined

In [16]:
early_stop=EarlyStopping(patience=5)
reducelr=ReduceLROnPlateau(patience=3)

### Model Architecture

In [17]:
suicide_model=Sequential()
suicide_model.add(Input(shape=(50,)))
suicide_model.add(Embedding(v+1,300,weights=[embedding_matrix],trainable=False))
suicide_model.add(LSTM(20,return_sequences=True))
suicide_model.add(GlobalMaxPooling1D())
#     model.add(Dropout(0.3))
suicide_model.add(Dense(256,activation='relu'))
#     model.add(Dropout(0.2))
suicide_model.add(Dense(1,activation='sigmoid'))
suicide_model.compile(optimizer=keras.optimizers.SGD(0.05,momentum=0.09),loss='binary_crossentropy',metrics=['accuracy'])

### Model training

In [19]:
r=suicide_model.fit(train_text_pad,train_output,validation_data=(test_text_pad,test_output),
            epochs=20,batch_size=64,callbacks=[early_stop,reducelr])

Epoch 1/20
1021/1021 [==============================] - 39s 38ms/step - loss: 0.1111 - accuracy: 0.9564 - val_loss: 0.1742 - val_accuracy: 0.9316 - lr: 0.0050
Epoch 2/20
1021/1021 [==============================] - 40s 39ms/step - loss: 0.1106 - accuracy: 0.9567 - val_loss: 0.1787 - val_accuracy: 0.9305 - lr: 0.0050
Epoch 3/20
1021/1021 [==============================] - 39s 39ms/step - loss: 0.1101 - accuracy: 0.9568 - val_loss: 0.1791 - val_accuracy: 0.9305 - lr: 0.0050
Epoch 4/20
1021/1021 [==============================] - 39s 38ms/step - loss: 0.1096 - accuracy: 0.9571 - val_loss: 0.1799 - val_accuracy: 0.9306 - lr: 0.0050
Epoch 5/20
1021/1021 [==============================] - 38s 37ms/step - loss: 0.1087 - accuracy: 0.9573 - val_loss: 0.1769 - val_accuracy: 0.9307 - lr: 5.0000e-04
Epoch 6/20
1021/1021 [==============================] - 33s 33ms/step - loss: 0.1086 - accuracy: 0.9574 - val_loss: 0.1778 - val_accuracy: 0.9307 - lr: 5.0000e-04


In [20]:
suicide_model.save('my_model.h5')

In [22]:
suicide_model = load_model('my_model.h5')

In [23]:
def detectSuicide(text):
    cleaned_text, length = clean_text(text)
    tokens = tokenizer.texts_to_sequences(cleaned_text)
    paded = pad_sequences(tokens,maxlen=50)
    if suicide_model.predict([paded]) >=0.50:
#         print("contains suicidal thoughts")
#         print(suicide_model.predict([paded]))
        return 0
    else:
#         print("Doesn't contains suicidal thoughts")
#         print(suicide_model.predict([paded]))
        return 1
detectSuicide(["I don't want to live anymore"])

100%|██████████| 1/1 [00:00<?, ?it/s]


1/1 [==============================] - 1s 635ms/step


0

In [37]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


### BERT Transformer for sentimental analysis

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel

In [26]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
class Transformer(nn.Module):
    def __init__(self, num_classes):
        super(Transformer, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.fc = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = bert_output[1]
        logits = self.fc(pooled_output)
        return logits

In [27]:
model = Transformer(num_classes=7)
model.load_state_dict(torch.load("model.pth", map_location= torch.device("cpu")))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [28]:
text = "The only reason I haven’t killed myself yet is because of my mom and brothers. If I didn’t have them I’d be dead long ago. "
tokens = bert_tokenizer.encode_plus(
    text,
    max_length=512,
    
    add_special_tokens=True,
    return_token_type_ids=False,
    padding='max_length',
    return_attention_mask=True,
    return_tensors='pt',
)
output = model((tokens['input_ids']), (tokens['attention_mask']))
predicted_label = torch.argmax(output, dim=1).item()
print(predicted_label)

0


In [29]:
detectSuicide(["i do not want to live anymore"])

100%|██████████| 1/1 [00:00<00:00, 1002.22it/s]


1/1 [==============================] - 0s 256ms/step


0

In [31]:
suicide_model.save("model_2.h5")

In [27]:
pip install pymongo 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
